In [ ]:
from bs4 import BeautifulSoup, NavigableString, Tag

import pathlib
import re

import pandas as pd

import db_connection.py

In [ ]:
# connect to database
cursor, connection = db_connection.get_connection()

In [3]:
data_path = str(pathlib.Path().resolve())

In [5]:
def remove_whitespace_in_id(match):
    id_value = match.group(1)
    cleaned = ''.join(id_value.split())  # Remove all whitespace
    return f'id="{cleaned}"'

In [ ]:
def amend_database_entry(cursor, connection, id, amended_text):
    sql = f"UPDATE e00_orgelpredigten SET e00volltext = '{amended_text}' WHERE e00id = '{id}'"
    cursor.execute(sql)
    connection.commit()
    print(cursor.rowcount, "record(s) affected") 

In [ ]:
with open("predigt_verbessert.txt", "r") as f:
    text = f.read()
    amend_database_entry(cursor, connection, "E000096", text.replace("'", "''"))

1 record(s) affected


In [ ]:
cursor.execute("""SELECT  e00id, e00volltext
                    FROM e00_orgelpredigten""")
sermons = cursor.fetchall()

In [18]:
def cleanup_sermon(text: str) -> str:
      """Cleans up organ sermon text and returns barebones xml.

      The text is stripped of all editorial and layout information.
      All tags are removed, apart from the following:
            * musikwerk: A direkt quote from a song
            * quelle: A quote from an authority
            * literatur: A quote from literature
            * bibel: A quote from the bible
            * quote: General quoted passages
            * orgelpredigt: A quote from another organ sermon
      The text is returned within <xml></xml>-tags, thus hopefully
      turning it into well-formed XML.

      Args:
          text (str): The input text

      Returns:
          str: The cleaned up text.
      """

      rep = {"<lb />": "", 
            "\r": "",
            #"\n": " ",
            "ᵜ": " ",
            "ʬ": "&",
            "<p>": "",
            "</p>": "",
            "<sic>": "",
            "</sic>": "",
            "<choice>": "",
            "</choice>": "",
            "</span>": "",
            "<div>": "",
            "</div>": "",
            "<err>": "",
            "</err>": "",
            "<fn></fn>": "",
            "<fn />": "",
            'typ="real"': "",
            '<?xml version="1.0" encoding="UTF-8"?>': ""
            }

      # use these three lines to do the replacement
      rep = dict((re.escape(k), v) for k, v in rep.items()) 
      pattern = re.compile("|".join(rep.keys()))
      text = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)

      text = re.sub(r'<pb page="[^"]+" ?\/?>(<\/pb>)?', "", text)              # remove page breaks
      text = re.sub(r'<h[123456]>([\S\s]+?)<\/h[123456]>', "", text)           # remove header tags

      text = re.sub(r'<x?person id="[^"]*">([\S\s]+?(?=<))<\/x?person>', r"\1", text)              # remove persons
      text = re.sub(r'<x?ort id="[^"]*">([\S\s]+?(?=<))<\/x?ort>', r"\1", text)                    # remove places
      text = re.sub(r'<x?orgel id="[^"]*">([\S\s]+?(?=<))<\/x?orgel>', r"\1", text)                # remove organ
      text = re.sub(r'<kunstwerk id="[^"]*">([\S\s]+?(?=<))<\/kunstwerk>', r'\1', text)            # remove kunstwerk
      text = re.sub(r'<ereignis id="[^"]*">([\S\s]+?(?=<))<\/ereignis>', r'\1', text)              # remove ereignis

      text = re.sub(r'<titel id="([^"]*)"( typ="real")? *>([\S\s]+?(?=<))</titel>', r'<quelle id="\1">\3</quelle>', text)

      text = re.sub(r'<supplied>[\S\s]+?<\/supplied>', "", text)                    # remove editorial additions
      text = re.sub(r'<corr>[\S\s]+?<\/corr>', "", text)                            # remove editorial corrections
      text = re.sub(r'<ref typ="trl">[\S\s]+?<\/ref>', "", text)                    # remove translations
      text = re.sub(r'<ref typ="anm">([\S\s]+?(?=<\/ref>))<\/ref>', "", text)       # remove editorial comments
      text = re.sub(r'<hi lang="[a-z]+">([\S\s]+?)<\/hi>', r"\1", text)             # remove typographical markup
      text = re.sub(r'(<hi rend="[a-z\-]+">)+([\S\s]+?)(<\/hi>)+', r"\2", text)     # remove typographical markup
      text = re.sub(r'<\/hi>', "", text)
      text = re.sub(r'<ref typ="ofn" symbol="\(\S+\)">', '', text)
      text = re.sub(r'<\/ref>', "", text)

      text = re.sub(r'<table([\S\s]+?(?=<))<\/table>', "", text)                          # remove tables

      text = re.sub(r'<div class="[a-z]+">', "", text)                                    # unwrap all <div> tags
      text = re.sub(r'=\s+', "", text)                                                    # undo hyphenations
      text = re.sub(r'<note>[\S\s]+?<\/note>', "", text)                                  # remove marginal notes
      text = re.sub(r'<span[^>]+>', "", text)

      text = re.sub(r'[\s\n]+', ' ', text)                                                # collapse all whitespace
      text = re.sub(r'(["\'\(\)»«›‹\.,\;])<', r'\1 <',text)
      text = re.sub(r'>(["\'\(\)»«›‹\.,\;])', '> \1', text)
      text = re.sub(r'id="([^"]+)"', remove_whitespace_in_id, text)                       # remove whitespace in IDs
      text = re.sub(r' </', '</', text)                                                   # remove whitespace before closing tags
      text = re.sub(r'\x01', '', text)
      
      return "<xml> " + text + " </xml>"

In [8]:
def remove_quotes(soup):
    # find all quote tags that are inside other tags or contain other tags and remove them
    for quote in soup.find_all("quote"):
        # If <quote> has children tags or is nested inside another tag (not direct child of root)
        has_nested_tags = any(child.name for child in quote.children)
        is_nested_in_another_tag = quote.parent.name != "root"

        if has_nested_tags or is_nested_in_another_tag:
            quote.unwrap()
    
    return soup

In [51]:
def soup_to_table(text: str) -> list:
    """Takes an xml string and turns it into a table of words and attributes

    Args:
        text (str): The text to be searched
    
    Returns:
        list: A table with the columns 'word', 'types', and 'reference'
    """

    return_table = []
    
    status = {
        "bibel": False,
        "quote": False,
        "quelle": False,
        "musikwerk": False,
        "literatur": False,
        "orgelpredigt": False
        }

    reference = []
    typemarker = ""

    openingtag = re.compile('<[^/]')
    irrelevantid = re.compile('id="E0[1234567][0-9]+"')

    words = re.split(r'[> ]+', text)

    for word in words:
        # update type assignment
        typemarker = ""
        for k, i in status.items():
            if i:
                typemarker = " ".join([typemarker, k])

        if re.match(openingtag, word):
            if word.startswith(("<xml", "</xml")):
                continue
            elif word.startswith("<bibel"):
                status["bibel"] = True
                continue
            elif word.startswith("<quote"):
                status["quote"] = True
                continue
            elif word.startswith("<quelle"):
                status["quelle"] = True
                continue
            elif word.startswith("<musikwerk"):
                status["musikwerk"] = True
                continue
            elif word.startswith("<literatur"):
                status["literatur"] = True
                continue
            elif word.startswith("<predigt"):
                status["orgelpredigt"] = True
                continue
            else:                               # ignore other stray tag fragments
                continue

        elif "</" in word:
            if word.endswith("</bibel"):
                status["bibel"] = False
                row = (word[:-7], typemarker, reference)
                reference = reference[:-1]
            elif word.endswith("</quote"):
                status["quote"] = False
                row = (word[:-7], typemarker, reference)
                reference = reference[:-1]
            elif word.endswith("</quelle"):
                status["quelle"] = False
                row = (word[:-8], typemarker, reference)
                reference = reference[:-1]
            elif word.endswith("</musikwerk"):
                status["musikwerk"] = False
                row = (word[:-11], typemarker, reference)
                reference = reference[:-1]
            elif word.endswith("</literatur"):
                status["literatur"] = False
                row = (word[:-11], typemarker, reference)
                reference = reference[:-1]
            elif word.endswith("</predigt"):
                status["orgelpredigt"] = False
                row = (word[:-9], typemarker, reference)
                reference = reference[:-1]
            else:
                row = (word.split("</")[0], typemarker, reference)
        
        elif word.startswith("id="):
            if re.match(irrelevantid, word):
                continue
            else:
                refs = re.findall('"(\S*?)"', word)
                if refs:
                    reference = reference + refs
                continue

        else:
            row = (word, typemarker, reference)
        return_table.append(row)

    return return_table

<>:93: SyntaxWarning: invalid escape sequence '\S'
<>:93: SyntaxWarning: invalid escape sequence '\S'
/tmp/ipykernel_3728/2401908148.py:93: SyntaxWarning: invalid escape sequence '\S'
  refs = re.findall('"(\S*?)"', word)


In [ ]:
# note: E000031 should be excepted since theres no markup!
for k, i in  enumerate(sermons):
    predigtid = i[0]
    predigttext = i[1]
    if predigtid != "E000031":
        print(f"Nr: {k}, predigtid={predigtid}")

        predigttext_cleaned = cleanup_sermon(predigttext)
        soup = BeautifulSoup(predigttext_cleaned, 'html.parser')
        soup = remove_quotes(soup)
        text = soup_to_table(re.sub(r"\x01", "", str(soup)))
        if len(text) > 500:
            df = pd.DataFrame(text, columns=["word", "types", "reference"])
            df.to_csv(f"sermon_tables/{predigtid}.tsv", sep="\t")

Nr: 0, predigtid=E000001
Nr: 1, predigtid=E000002
Nr: 2, predigtid=E000003
Nr: 3, predigtid=E000004
Nr: 4, predigtid=E000005
Nr: 5, predigtid=E000006
Nr: 6, predigtid=E000007
Nr: 7, predigtid=E000008
Nr: 8, predigtid=E000009
Nr: 9, predigtid=E000010
Nr: 10, predigtid=E000011
Nr: 11, predigtid=E000012
Nr: 12, predigtid=E000013
Nr: 13, predigtid=E000014
Nr: 14, predigtid=E000015
Nr: 15, predigtid=E000016
Nr: 16, predigtid=E000017
Nr: 17, predigtid=E000018
Nr: 18, predigtid=E000019
Nr: 19, predigtid=E000020
Nr: 20, predigtid=E000021
Nr: 21, predigtid=E000022
Nr: 22, predigtid=E000023
Nr: 23, predigtid=E000024
Nr: 24, predigtid=E000025
Nr: 25, predigtid=E000026
Nr: 26, predigtid=E000027
Nr: 27, predigtid=E000028
Nr: 28, predigtid=E000029
Nr: 29, predigtid=E000030
Nr: 31, predigtid=E000032
Nr: 32, predigtid=E000033
Nr: 33, predigtid=E000034
Nr: 34, predigtid=E000035
Nr: 35, predigtid=E000036
Nr: 36, predigtid=E000037
Nr: 37, predigtid=E000038
Nr: 38, predigtid=E000039
Nr: 39, predigtid=E000